# Imports

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# !cp '/content/drive/MyDrive/dermoscopy_classification.tar.gz' .
# !tar -xvzf 'dermoscopy_classification.tar.gz'
data_dir = '/home/ilias/Desktop/Ergasies/ml/2nd_assignment/dermoscopy_classification'

In [2]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from glob import glob
import os
from torch.utils.data import random_split
from torch import nn, optim
import matplotlib.pyplot as plt

# 2.1-2.2 Μέθοδοι __init__(), __len__(), __getitem__()

In [3]:
class MLProject2Dataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, metadata_fname='/home/ilias/Desktop/Ergasies/ml/2nd_assignment/dermoscopy_classification/metadata.csv', transform=None):

        self.data_dir = data_dir
        self.transform = transform

        # Find all images in the data directory
        image_paths = []
        for part in ["part_1", "part_2"]:
            part_path = os.path.join(data_dir, part)
            image_paths += glob(os.path.join(part_path, "*.jpg"))

        # Create a DataFrame with the image paths
        df = pd.DataFrame({"path": image_paths, "image_id": [os.path.splitext(os.path.basename(p))[0] for p in image_paths]})

        # Load the metadata
        metadata_path = os.path.join(data_dir, metadata_fname)
        metadata = pd.read_csv(metadata_path)

        # Convert DX column to integers
        metadata['dx'] = pd.Categorical(metadata['dx']).codes

        # Combine df and dx label into a pandas DataFrame
        df = pd.merge(df, metadata[['image_id', 'dx']], on='image_id')
        self.df = df


    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):

        img_path = self.df.iloc[idx, 0]  # path is in 1st column
        label = self.df.iloc[idx, 2]  # dx is in 3rd column
        image = torchvision.io.read_image(img_path).float() / 255.0  # Read image and normalize pixel values to [0-1]

        X = image
        y = label
        return (X, y)

# 2.3 Train / validation / test split και Data loaders

In [4]:
dataset = MLProject2Dataset(data_dir)

# Define the sizes for train, validation, and test sets
dataset_size = len(dataset)
train_size = int(0.6 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Use the random_split method with a fixed seed
generator = torch.Generator()
generator.manual_seed(42)

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size], generator=generator)

batch_size=64
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_set, batch_size=batch_size)
testloader = DataLoader(test_set, batch_size=batch_size)

# 2.4 Μετασχηματισμοί



In [5]:
# Define the resize and normalization transformations
m, n = 224, 224  # Define your desired dimensions m x n
resize = transforms.Resize((m, n))
normalize = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))

# Combine transformations for the dataset
composed_transforms = transforms.Compose([resize, transforms.ToTensor(), normalize])

# Apply transformations to train_set, val_set, test_set
train_set.transform = composed_transforms
val_set.transform = composed_transforms
test_set.transform = composed_transforms

# 3.1 train_net

In [7]:
def train_net(model: nn.Module, trainloader: DataLoader, valloader: DataLoader = None,
              epochs: int = 10, optimizer: optim = None, loss: nn.modules.loss = None,
              device: str = 'cpu', print_period: int = 10) -> None:

      model.train()

      for epoch in range(epochs):
          running_loss = 0.0

          for batch, (X, y) in enumerate(trainloader, 0):
              # Move to device
              X, y = X.to(device), y.to(device)

              # Zero gradients
              optimizer.zero_grad()

              # Forward pass
              pred = model(X)
              current_loss = loss(pred, y)

              # Αccuracy
              preds = pred.argmax(dim=1)
              corrects = preds == y
              running_corrects += corrects.sum().item()

              # Backpropagation
              current_loss.backward()

              # Parameter update
              optimizer.step()

              # Print
              running_loss += current_loss.item()
              if batch % print_period == print_period-1:
                  avg_loss = running_loss / print_period
                  print(f'[Epoch: {epoch}, batch: {batch:5d}] loss: {avg_loss:.3f}, acc: {running_corrects / len(trainloader.dataset):.4f}')
                  running_loss = 0.0

                  if valloader is not None:
                      val_loss = 0.0
                      val_correct = 0
                      val_total = 0

                      model.eval()
                      with torch.no_grad():
                          for data in valloader:
                              inputs, labels = data[0].to(device), data[1].to(device)
                              outputs = model(inputs)
                              val_loss += loss(outputs, labels).item()

                              _, predicted = torch.max(outputs, 1)
                              val_total += labels.size(0)
                              val_correct += (predicted == labels).sum().item()

                      print(f'Validation Loss: {val_loss / len(valloader):.3f}, '
                            f'Validation Accuracy: {(val_correct / val_total) * 100:.2f}%')

# 3.2 test_net

In [8]:
def test_net(model: nn.Module, testloader: torch.utils.data.DataLoader,
             loss_fn: nn.modules.loss = None, device: str = 'cpu') -> None:

    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for (X, y) in testloader:
            inputs, labels = X.to(device), y.to(device)
            outputs = model(inputs)
            test_loss += loss_fn(outputs, labels).item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    average_loss = test_loss / len(testloader)

    print(f'Test Loss: {average_loss:.3f}, Test Accuracy: {accuracy * 100:.2f}%')

    return average_loss, accuracy

# 4 Απλό ΣΝ∆

In [10]:
class MyCNN(nn.Module):
    def __init__(self):

        super(MyCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(268800, 64)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [11]:
# Επανορίζουμε το μοντέλο μας και τον ορισμό της απώλειας και του optimizer
model = MyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Λίστες για την καταγραφή των μετρήσεων
train_losses = []
train_accuracies = []

# Εκπαίδευση του μοντέλου για 20 εποχές
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0], data[1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(trainloader)
    epoch_accuracy = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy * 100:.2f}%")

# Αξιολόγηση του μοντέλου στο σύνολο δοκιμής
test_loss, test_accuracy = test_net(model, testloader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")

# Απεικόνιση των μετρήσεων απώλειας και ευστοχίας κατά την εκπαίδευση
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

RuntimeError: expected scalar type Long but found Char